In [11]:
from rich import print, print_json

In [12]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader

In [13]:
model_nemo_a = ChatOllama(model="nemotron-mini:latest", keep_alive=True, temperature=0)
model_nemo_b = ChatOllama(
    model="nemotron-mini:4b-instruct-q8_0", keep_alive=True, temperature=0
)
model_embedding = OllamaEmbeddings(model="nomic-embed-text:latest")

In [25]:
vectore_store = Chroma(embedding_function=model_embedding)

In [15]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

loader = DirectoryLoader(
    "C:/Users/roshan.yadav/Obsidian/Github Projects Description/",
    glob="*.md",
    loader_cls=UnstructuredMarkdownLoader,
    show_progress=True,
    use_multithreading=True,
)

file = loader.load()

100%|██████████| 12/12 [00:00<00:00, 144.25it/s]


In [16]:
file

[Document(metadata={'source': 'C:\\Users\\roshan.yadav\\Obsidian\\Github Projects Description\\Auto Dictionary database creator tool.md'}, page_content='The goal of this project is to retrieve English words and their Hindi translations using Google Translate. Initially, I used the website www.mso.anu.edu.au to gather dictionary words from A to Z. The collected words, along with their definitions, were structured as key-value pairs and stored in a JSON file.\n\nTo automate the process, I initially created a PyAutoGUI script. The script manually opened a Chrome tab with Google Translate, iterated through the JSON file, and simulated Ctrl + C and Ctrl + V to input English words into the translation box. It then copied the Hindi translation by identifying the "copy" button using the locateOnScreen method. However, this approach was inefficient and made the computer unusable while the script was running.\n\nTo address these issues, I redeveloped the solution using Selenium, which significan

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [21]:
smaller_documents = text_splitter.split_documents(file)
smaller_documents

[Document(metadata={'source': 'C:\\Users\\roshan.yadav\\Obsidian\\Github Projects Description\\Auto Dictionary database creator tool.md'}, page_content='The goal of this project is to retrieve English words and their Hindi translations using Google Translate. Initially, I used the website www.mso.anu.edu.au to gather dictionary words from A to Z. The collected words, along with their definitions, were structured as key-value pairs and stored in a JSON file.\n\nTo automate the process, I initially created a PyAutoGUI script. The script manually opened a Chrome tab with Google Translate, iterated through the JSON file, and simulated Ctrl + C and Ctrl + V to input English words into the translation box. It then copied the Hindi translation by identifying the "copy" button using the locateOnScreen method. However, this approach was inefficient and made the computer unusable while the script was running.\n\nTo address these issues, I redeveloped the solution using Selenium, which significan

In [ ]:
data = vectore_store.add_documents(documents=smaller_documents)
data

['66e86a53-822b-4f40-a1d5-6affc0508cff',
 'ce9a5ffb-00b7-49cd-aa5c-aca03039666d',
 'e8cfa00e-10a3-4412-83c6-83fe57ef5972',
 'e3d08531-e6fc-460f-8980-2e33bc499fad',
 '6241c307-729a-42f1-8e86-1ce565352011',
 'a7139c31-3bba-4f77-a348-5c195980e52b',
 '174fff8c-3e8f-40d7-8622-02898dd6fe43',
 'a22db673-e667-4920-a4cd-adaa94e97835',
 '1594904d-526f-4348-9966-b020f2414864',
 'e2e0a5c7-4323-4b97-a147-e9ca4c341c0d',
 '173895f9-ab73-4caa-a0d5-f8e5e8ca56de',
 '8929570e-37e7-446b-a849-b78732b0b6d2']

In [30]:
vectore_store.similarity_search(
    "what is the goal of tool 'Stack Overflow data Analysis'?", k=3
)

[Document(metadata={'source': 'C:\\Users\\roshan.yadav\\Obsidian\\Github Projects Description\\Stack Overflow data Analysis.md'}, page_content='This tool is developed to analyze data published on Stack Overflow, allowing it to categorize the information by age group. It identifies which programming languages are most popular within each group, what new languages they plan to learn in the future, and the sources they rely on for learning.'),
 Document(metadata={'source': 'C:\\Users\\roshan.yadav\\Obsidian\\Github Projects Description\\Telegram Views and reaction Booster.md'}, page_content='A tool designed to boost views on Telegram channel posts by utilizing different IPs, including both datacenter and residential IPs. It also sends reactions to posts using various Telegram sessions generated with dummy numbers. This tool is used to build social proof for services, helping to accelerate their initial growth.'),
 Document(metadata={'source': 'C:\\Users\\roshan.yadav\\Obsidian\\Github Pro

In [31]:
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [32]:
from langgraph.graph import START, StateGraph


def retriever(state: State):
    return {"context": vectore_store.similarity_search(state["question"], k=3)}

In [33]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
)

In [34]:
def addDocumentinPromptAndQuery(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = model_nemo_a.invoke(messages)
    return {"answer": response.content}

In [35]:
graph_builder = StateGraph(State).add_sequence([retriever, addDocumentinPromptAndQuery])
graph_builder.add_edge(START, "retriever")
graph = graph_builder.compile()

In [1]:
response = graph.invoke({"question": "what is the goal of tool 'Stack Overflow data Analysis'?"})
print(response["answer"])

NameError: name 'graph' is not defined